## HILL CLIMBING

In [1]:
import numpy as np

#### Função Estado Inicial 

* Gerando matriz de dimensão NxN onde N é o número de rainhas e atribuindo uma posição para cada rainha.

In [2]:
def estadoInicial(numRainhas):
    
    tabuleiro = np.zeros((numRainhas, numRainhas), 
                         dtype=np.int)
    
    for i in range(numRainhas):
        posicaoAleatoria = np.random.randint(numRainhas)
        tabuleiro[posicaoAleatoria, i] = 1
        
    return tabuleiro

#### Função Diagonal
* Função para percorrer diagonais da matriz do tabuleiro

In [3]:
def funcaoDiagonal(dimensao, valor, tipo='P'):
    
    linhas = []; colunas = []
    sinal  = 1 if tipo == 'P' else -1
    
    for i in range(dimensao):
        for j in range(dimensao):
            if i + j*sinal == valor:
                linhas.append(i)
                colunas.append(j)
                
    return linhas, colunas

#### Função Ataque Linhas
* Função para calcular o número total de ataques nas linhas do tabuleiro

In [4]:
def ataqueLinha(tabuleiro, dimensao, ataques=0):
    
    for linha in range(dimensao):
        numRainhas = np.count_nonzero(tabuleiro[linha, :])
        ataques += numRainhas - 1 if numRainhas > 1 else 0
    
    return ataques

#### Função Ataque Diagonal Primária
* Sabendo que LINHA + COLUNA resultam em constantes para uma determinada diagonal primária teremos: 
* 1 a 2N -3, onde N é parte da dimensão NxN da matriz (tabuleiro)

In [5]:
def ataqueDiagonalPrimaria(tabuleiro, dimensao, ataques=0):
    
    for valorDiagonal in range(1, 2*dimensao-3):
        linhas, colunas = funcaoDiagonal(dimensao, valorDiagonal)
        numRainhas = np.count_nonzero(tabuleiro[linhas, colunas])
        ataques += numRainhas - 1 if numRainhas > 1 else 0
        
    return ataques

#### Ataque Diagonal Secundária
* Sabendo que LINHA - COLUNA resultam em constantes para uma determinada diagonal secundária teremos: 
* -N+2 a N-1, onde N é parte da dimensão NxN da matriz (tabuleiro)

In [6]:
def ataqueDiagonalSecundaria(tabuleiro, dimensao, ataques=0):
    
    for valorDiagonal in range(-dimensao+2, dimensao-1):
        linhas, colunas = funcaoDiagonal(dimensao, valorDiagonal, tipo='S')
        numRainhas = np.count_nonzero(tabuleiro[linhas, colunas])
        ataques += numRainhas - 1 if numRainhas > 1 else 0
        
    return ataques

#### Função Custo
* Função para calcular o número total de pares de damas em ataque com base nas funções de ataque acima

In [16]:
def funcaoCusto(tabuleiro):

    dimensao = tabuleiro.shape[0]
    
    ataques  = ataqueLinha(tabuleiro, dimensao)
    ataques += ataqueDiagonalPrimaria(tabuleiro, dimensao)
    ataques += ataqueDiagonalSecundaria(tabuleiro, dimensao)
     
    return ataques


#### Função Matriz de Custo
* Utilizando a função anterior, é possível chegar na matriz de custo. Ou seja, calcularmos os custos para todos os movimentos possíveis das rainhas em suas colunas do tabuleiro.

In [17]:
def gerarmatrizCusto(tabuleiro):

    dimensao = tabuleiro.shape[0]
    custos   = []

    for coluna in range(dimensao):
        posicaoInicialRainha = tabuleiro[:, coluna].argmax()
        tabuleiro[:, coluna] = 0
        
        for linha in range(dimensao):
            tabuleiro[linha, coluna] = 1
            custos.append(funcaoCusto(tabuleiro))
            tabuleiro[linha, coluna] = 0
            
        tabuleiro[posicaoInicialRainha, coluna] = 1
        
    return np.array(custos).reshape((dimensao, dimensao)).T

### Funções de Mutação (Melhor Vizinho)
* Funções para tentativa de alcançar a função ótima, utilizando a lógica de "melhor vizinho" 

In [18]:
# POSIÇÕES DAS RAINHAS
def gerarPosicaoRainhas(tabuleiro):
    
    linhas, colunas = np.where(tabuleiro == 1)
    return set(zip(linhas, colunas))


In [20]:
# POSIÇÕES SEM RAINHAS
def gerarPosicaoAleatoria(tabuleiro):
    
    linhas, colunas = np.where(tabuleiro == 0)
    return list(zip(linhas, colunas))


In [21]:
# POSSÍVEIS MOVIMENTOS NO TABULEIRO  
def gerarMovimentos(matrizCusto):
    
    linhas, colunas = np.where(matrizCusto == np.min(matrizCusto))
    return set(zip(linhas, colunas))

In [22]:
# ITERAÇÕES DE MUDANÇA DE ESTADO NO TABULEIRO 
def mudarDeEstado(tabuleiro):

    dimensao = tabuleiro.shape[0]
    
    posicaoRainhas = gerarPosicaoRainhas(tabuleiro)
    matrizCusto  = gerarmatrizCusto(tabuleiro)
    possibilidades = gerarMovimentos(matrizCusto)
    
    if (possibilidades - posicaoRainhas) == set():
        print('\nSHOW! Temos um Mínimo Local!\n')
        movimentos = gerarPosicaoAleatoria(tabuleiro)
        
    else:
        movimentos = list(possibilidades - posicaoRainhas)
        
    aleatorio  = np.random.randint(len(movimentos))
    movimento  = movimentos[aleatorio]

    tabuleiro[:, movimento[1]] = 0
    tabuleiro[movimento]       = 1
    
    print('\nMatriz de Custo') 
    print(matrizCusto.view())
    
    return tabuleiro


### Algoritmo de Hill Clumbing

In [23]:
def hillClimbing(numRainhas):
    
    tabuleiro = estadoInicial(numRainhas)
    
    print('\nEstado Inicial') 
    print(tabuleiro.view())
    
    movimentos = 0
    while(funcaoCusto(tabuleiro)):
        tabuleiro = mudarDeEstado(tabuleiro)
        movimentos += 1
        
        print('\nMelhor Vizinho # %d' % movimentos) 
        print(tabuleiro.view())
        
        
    print('\nCusto:', funcaoCusto(tabuleiro))
    print('Total Movimentos: %d\n' % movimentos)

In [15]:
%time hillClimbing(numRainhas=8)


Estado Inicial
[[0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 1 0]
 [0 0 1 0 0 0 0 0]]

Matriz de Custo
[[8 7 7 7 8 7 9 9]
 [7 8 7 8 8 8 9 8]
 [6 6 8 6 8 7 8 8]
 [8 8 7 9 9 7 9 8]
 [7 7 7 7 8 7 8 7]
 [7 7 8 6 8 8 7 8]
 [8 8 7 8 9 7 8 8]
 [8 8 8 8 9 8 8 9]]

Melhor Vizinho # 1
[[0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0]]

Matriz de Custo
[[6 6 6 7 6 6 7 7]
 [5 6 7 8 7 6 7 6]
 [5 5 8 6 7 6 7 7]
 [5 6 7 9 8 5 7 6]
 [5 5 6 7 6 6 6 5]
 [6 5 6 6 6 6 6 6]
 [6 6 6 8 7 5 6 7]
 [6 6 6 8 6 6 6 7]]

Melhor Vizinho # 2
[[0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 1 0 0 0 1 1 0]
 [0 0 1 0 0 0 0 0]]

Matriz de Custo
[[6 5 5 6 5 6 6 6]
 [5 4 5 6 5 6 5 4]
 [5 5 7 5 6 6 6 6]
 [5 5 7 8 7 5 6 5]
 [5 4 5 7 5 6 5 5]
 [6 4 5 5 6 6 6 5]
 [5 5 5 7 6 5 